## IBSI Chapter 1 Phase 2 − Radiomic Computations

@Author : [MEDomics consortium](https://github.com/medomics/)

@EMAIL : medomics.info@gmail.com

@REF : [IBSI 1](https://arxiv.org/pdf/1612.07003.pdf)

**STATEMENT**:
This file is part of <https://github.com/MEDomics/MEDomicsLab/>,
a package providing PYTHON programming tools for radiomics analysis.
--> Copyright (C) MEDomicsLab consortium.

### Introduction

We recommend to take a look at the [MEDimage-tutorial notebook](https://colab.research.google.com/github/MahdiAll99/MEDimage/blob/dev/notebooks/tutorial/MEDimage-Tutorial.ipynb) and the [DataManager-tutorial](https://colab.research.google.com/github/MahdiAll99/MEDimage/blob/dev/notebooks/tutorial/DataManager-Tutorial.ipynb) before running the IBSI tests.

In this notebook we treat the second phase where we focus on the standardization of image processing and feature computation. In the figure below, we focus on the second part, phase 2 where we compute radiomics features from a CT image after certain processing.

<img src="images/Flowchart-of-study-overview.png" alt="range resegmentation example"/>


### Dataset - CT image
In this chapter and in this phase, reference values for features were obtained using a CT image, which is described below. The CT image can be found here: [IBSI 1 CT Phantom](https://github.com/theibsi/data_sets/tree/master/ibsi_1_ct_radiomics_phantom/dicom)

The CT image set is stored as a stack of slices in DICOM format. The gross tumour volume (GTV) was delineated and is used as the region of interest (ROI). Contour information is stored as an RT structure set in the DICOM
file starting with DCM RS. For broader use, both the DICOM set and segmentation mask have been
converted to the NIfTI format. When using the data in NIfTI format, both image stacks should
be converted to (at least) 32-bit floating point and rounded to the nearest integer before further
processing.

In [1]:
import os
import sys

MODULE_DIR = os.path.dirname(os.path.abspath('../MEDimage/'))
sys.path.append(os.path.dirname(MODULE_DIR))

from copy import deepcopy
from pathlib import Path

import MEDimage

### Configuration

In this phase, we test many configurations, each configuration has different parameters, algorithms etc. Choosing what configuration is used for image processing is very important. The figure below shows the different configurations possible: 

*A, B, C ,D and E*.

In [2]:
CONFIG = 'E' # script configuration. Possible values : 'A', 'B', 'C', 'D', 'E'

<img src="images/ibsi1-p1-configs.png" alt="Flowchart of radiomics study"/>

Refer to the [IBSI 1 Reference chapter 5](https://arxiv.org/pdf/1612.07003.pdf#chapter.5) for more details.

### Classes initialization

In the IBSI scripts we are going to use the *MEDimage* class and other processing classes like ``DataManager`` to process the imaging data and to extract the radiomics features.
- ``MEDimage``: Is a Python class that organizes all scan data and many other useful information that is used by the many processing and computing methods.
- ``DataManager``: A Python class that process all the raw data (DICOM and NIfTI) and convert it to *MEDimage* class objects.

So the first step is to initialize the MEDimage class using the ``DataManager``. This class will only need path to where the data is located (either DICOM or NIfTI). We will use DICOM for now.

Make sure your folder structure looks like this:

<img src="images/ibsi1-p2-folder-structure.png"/>

Make sure to also download the [CT-Image data](https://github.com/theibsi/data_sets/tree/master/ibsi_1_ct_radiomics_phantom/dicom) (DICOM files) and place it in CTimage folder. The settings file can be found in the repository and is automatically place in the settings folder.

We start by initializing the important paths to settings folder, dataset folder...

In [3]:
path_settings = Path(os.getcwd()) / "settings" # Path to the script settings/configuration folder

# Load script parameters
im_params = MEDimage.utils.json_utils.load_json(path_settings / f'Phase2{CONFIG}_settings.json')

Then we'll initialization the ``DataManager`` class and run DICOMs processing

In [4]:
# Intialize the DataManager class
path_to_dicoms = Path(os.getcwd()) / "data" / "CTimage"
dm = MEDimage.wrangling.DataManager(path_to_dicoms=path_to_dicoms)

# Process the DICOM scan
MEDinstance = dm.process_all_dicoms()[0]  # one-element list/ one scan

2022-08-15 18:07:18,077	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8267


--> Reading all DICOM objects to create MEDimage classes

--> Scanning all folders in initial directory...

100%|██████████| 61/61 [00:00<00:00, 3334.84it/s]


DONE
--> Associating all RT objects to imaging volumes


100%|██████████| 1/1 [00:00<00:00, 32017.59it/s]


DONE


100%|██████████| 1/1 [00:00<00:00, 7358.43it/s]

DONE


### Image processing
Unlike phase 1, processing is required here and different configurations can be tested as mentionned in the IBSI chapter 1. The processing steps are:
- **Segmentation (Creation of ROI mask)**
- **Interpolation**
- **Re-segmentation**
- **ROI extraction**
- **Intensity discretization**


#### Segmentation

ROIs are used to define the region in which features are calculated. What constitutes an ROI depends on the imaging and the study objective. In medical imaging of cancer patients, the tumour volume is a common ROI. ROIs can be defined manually by experts or (semi-)automatically using algorithms. ROIs are typically stored with the accompanying image (which is the case here).

First, we extract the ROI mask

In [5]:
vol_obj_init, roi_obj_init = MEDimage.processing.get_roi_from_indexes(MEDinstance,
                                                           name_roi='{GTV-1}',
                                                           box_string='full')

Initialize processing & computation parameters

In [6]:
MEDinstance.init_params(im_params)

We extract the initial diagnostic features, so we can identify if there is any issues with the implementation before image processing.

In [7]:
# intial diagnostic features extraction:
diag_initial = MEDimage.biomarkers.diagnostics.extract_all(
                                vol_obj_init, 
                                roi_obj_init, 
                                roi_obj_init, 
                                'initial')

# Update diagnostic Dict:
MEDinstance.radiomics.image.update(
    {'Diagnostics-initial': 
        {'scale' + str(MEDinstance.params.process.scale_non_text[0]): diag_initial}
    })

#### Interpolation

Texture feature sets require interpolation to isotropic voxel spacing to be rotationally invariant, and to allow comparison between image data from different samples, cohorts or batches. Interpolation algorithms translate image intensities from the original image grid to an interpolation grid. 

The figure below shows different interpolation mesh grids based on an original 4 × 4 grid with (3.00, 3.00) mm spacing. The desired interpolation spacing is (2.00, 2.00) mm. *Fit to original grid* creates an interpolation mesh grid that overlaps with the corners of the original grid. *Align grid* origins creates an interpolation mesh grid that is positioned at the origin of the original grid. *Align grid centers* creates an interpolation grid that is centered on the center of original and interpolation grids

<img src="images/interpolation-ex.png" alt="Flowchart of radiomics study"/>

We interpolate both volume data and ROI data in order to create the morphological and the intensity masks:

   - **The morphological mask**: is not re-segmented and maintains the original morphology as defined by an expert and/or (semi-)automatic segmentation algorithms.
   - **The intensity mask**: can be re-segmented and will only contain the selected voxels. It is important for many feature families. 

**PS** : Interpolation is not required for the configuration A.

In [8]:
vol_obj = deepcopy(vol_obj_init)
roi_obj_morph = deepcopy(roi_obj_init)

if CONFIG != 'A':
    # Intensity Mask
    vol_obj = MEDimage.processing.interp_volume(
        MEDimage=MEDinstance,
        vol_obj_s=vol_obj_init,
        vox_dim=MEDinstance.params.process.scale_non_text,
        interp_met=MEDinstance.params.process.vol_interp,
        round_val=MEDinstance.params.process.gl_round,
        image_type='image',
        roi_obj_s=roi_obj_init,
        box_string='full'
    )
    # Morphological Mask
    roi_obj_morph = MEDimage.processing.interp_volume(
        MEDimage=MEDinstance,
        vol_obj_s=roi_obj_init,
        vox_dim=MEDinstance.params.process.scale_non_text,
        interp_met=MEDinstance.params.process.roi_interp,
        round_val=MEDinstance.params.process.roi_pv,
        image_type='roi', 
        roi_obj_s=roi_obj_init,
        box_string='full'
    )

We can see how the voxel spacing can change before interpolation and after interpolation (If config is different than A)

In [9]:
# Desired voxel spacing
print(f'Desired spacing (Config {CONFIG}):', 
      MEDinstance.params.process.scale_non_text)

# voxel spacing before interpolation
print('Before interp:', [vol_obj_init.spatialRef.PixelExtentInWorldX, 
      vol_obj_init.spatialRef.PixelExtentInWorldY,
      vol_obj_init.spatialRef.PixelExtentInWorldZ])

# voxel spacing after interpolation
print('After interp:',[vol_obj.spatialRef.PixelExtentInWorldX, 
      vol_obj.spatialRef.PixelExtentInWorldY,
      vol_obj.spatialRef.PixelExtentInWorldZ])

Desired spacing (Config E): [2, 2, 2]
Before interp: [0.977, 0.977, 3.0]
After interp: [2, 2, 2]


Post-interpolation diagnostic features computation

In [10]:
# Post-interpolation diagnostic features extraction:
diag_interp = MEDimage.biomarkers.diagnostics.extract_all(vol_obj, roi_obj_morph, roi_obj_morph, 'interp')

# Update diagnostic Dict:
MEDinstance.radiomics.image.update(
    {'Diagnostics-interpolated': 
        {'scale' + str(MEDinstance.params.process.scale_non_text[0]): diag_interp}
    })

#### Re-segmentation

Next processing step is re-segmentation, we update the ROI mask based on corresponding configuration given voxel intensities (For example exclusion of air or bone voxels from a defined ROI on CT image). 

- **range re-segmentation**

Range re-segmentation will remove voxels from the intensity mask that fall outside of a specified range. 

The figure below shows how intensity and morphological masks change with re-segmentation. (1) The original region of interest (ROI) is shown with pixel intensities. (2) Subsequently, the ROI is re-segmented to only contain values in the range [1, 6]. Pixels outside this range
are marked for removal from the intensity mask. (3a) Resulting morphological mask, which is identical
to the original ROI. (3b) Re-segmented intensity mask.

<img src="images/Example-resegmentation.png" alt="range resegmentation example" style="width:500px;"/>

In [11]:
# Intensity mask re-segmentation
roi_obj_int = deepcopy(roi_obj_morph)

roi_obj_int.data = MEDimage.processing.range_re_seg(
            vol=vol_obj.data, 
            roi=roi_obj_int.data,
            im_range=MEDinstance.params.process.im_range
        )

- **outlier re-segmentation**

ROI voxels with outlier intensities are removed from the intensity mask.

The figure below shows a re-segmentation example based on a CT-image. The masked region (blue) is re-
segmented to create an intensity mask (orange). Examples using three different re-segmentation parameter sets are shown. The bottom right combines the range and outlier re-segmentation, and the
resulting mask is the intersection of the masks in the other two examples

<img src="https://ibsi.readthedocs.io/en/latest/_images/resegmentation.png" alt=" outlier resegmentation example" style="width:500px;"/>

In [12]:
import numpy as np


# re-compute the range (externally) for comparison
u = np.mean(vol_obj.data[roi_obj_int.data == 1])
sigma = np.std(vol_obj.data[roi_obj_int.data == 1])

outlier_range = [u + 3*sigma, u - 3*sigma]

# Intensity mask outlier re-segmentation
roi_obj_int.data = np.logical_and(MEDimage.processing.outlier_re_seg(
            vol=vol_obj.data, 
            roi=roi_obj_int.data, 
            outliers=MEDinstance.params.process.outliers),
            roi_obj_int.data
).astype(int)

Post-re-segmentation diagnostic features computation

In [13]:
# Post-resegmentation diagnostic features extraction:
diag_re_seg = MEDimage.biomarkers.diagnostics.extract_all(vol_obj, roi_obj_int, roi_obj_morph, 'reSeg')

# Update diagnostic Dict:
MEDinstance.radiomics.image.update(
    {'Diagnostics-resegmented': 
            {'scale' + str(MEDinstance.params.process.scale_non_text[0]): diag_re_seg}
    })

#### ROI extraction

After re-segmentation, we extract the image volume that will be studied that is defined by the ROI intensity mask. Excluded voxels (Intensities outside ROI) will be replaced by *NaN* and voxels included in the ROI mask retain their original intensity as shows the figure below.

<img src="https://ibsi.readthedocs.io/en/latest/_images/roi_extraction.png" alt=" ROI extraction example" style="width:500px;"/>


In [14]:
vol_int_re = MEDimage.processing.roi_extract(
            vol=vol_obj.data, 
            roi=roi_obj_int.data
        )

We can check the change of intensity range before and after re-segmentation. The max and min should respect the re-segmentation range defined in the settings and the outlier range defined by the outlier re-seg algorithm. For example : in configuration A, the re-segmentation range is [-500, 400] so the intensities min and the intensities max should not be lower or higher than -500 and 400 respectively

In [15]:
# Desired re-segmentation range
print(f'Desired re-segmentation range (Config {CONFIG}): {MEDinstance.params.process.im_range}')

print(f'Desired re-segmentation outlier range (Config {CONFIG}): {outlier_range}')

# Intensity range before re-segmentation
print(f'Intensity range before, min:{np.min(vol_obj_init.data)} max:{np.max(vol_obj_init.data)}')

# Intensity range after re-segmentation
print(f'Intensity range after, min:{np.nanmin(vol_int_re)} max:{np.nanmax(vol_int_re)}')

Desired re-segmentation range (Config E): [-1000, 400]
Desired re-segmentation outlier range (Config E): [646.9928321838379, -743.6982383728027]
Intensity range before, min:-1000.0 max:3065.0
Intensity range after, min:-743.0 max:345.0


Initialization of all the computation parameters for NON-TEXTURE FEATURES.

In [16]:
# Preparation of computation:
MEDinstance.init_ntf_calculation(vol_obj)

### Non-Texture Features extraction
In this section we extract the following families of features using MEDimageComputeRadiomics methods : 

*morphological features, local intensity, statistical, Intensity-based and intensity histogram-based.*

Further image processing is required for some features.

#### Morphological features

Morphological features describe geometric aspects of a region of interest (ROI), such as area and
volume. Morphological features are based on ROI voxel representations of the volume.

In [17]:
morph = MEDimage.biomarkers.morph.extract_all(
            vol=vol_obj.data, 
            mask_int=roi_obj_int.data, 
            mask_morph=roi_obj_morph.data,
            res=MEDinstance.params.process.scale_non_text,
        )

#### Local intensity features

Voxel intensities within a defined neighborhood around a center voxel are used to compute local
intensity features. By definition, local intensity features are calculated in 3D, and not per slice.

In [18]:
local_intensity = MEDimage.biomarkers.local_intensity.extract_all(
            img_obj=vol_obj.data,
            roi_obj=roi_obj_int.data,
            res=MEDinstance.params.process.scale_non_text
        )

#### Intensity-based statistical features

The intensity-based statistical features describe how intensities within the region of interest (ROI)
are distributed. The features in this set do not require discretization, and may be used to describe
a continuous intensity distribution.

In [19]:
stats = MEDimage.biomarkers.stats.extract_all(vol=vol_int_re)

#### Intensity discretisation

As mentioned before, further processing is required for some non-texture features. The last processing step is intensity discretization. Two approaches can be used : Fixed bin size (FBS) or Fixed bin number (FBN).
- FBS : intensities are discretized to a fixed number of bins.
- FBN : intensities are discretized to a fixed size of bins.

The figure below shows Discretization of two different PET images with SUVmax of 27.8 (A) and 6.6 (B). *Fixed bin number* discretization adjust the contrast between the two images, with the number of bins determining the coarseness of the discretized image. *Fixed bin size discretization* leaves the contrast differences between image A and B intact. Increasing the bin size increases the coarseness of the discretized image.

<img src="https://ibsi.readthedocs.io/en/latest/_images/discretisation.png" alt=" discretization example" style="width:500px;"/>

Read more about discretization : [Intensity discretization](https://arxiv.org/pdf/1612.07003.pdf#section.2.7)

In [20]:
vol_quant_re, _ = MEDimage.processing.discretisation.discretize(
            vol_re=vol_int_re,
            discr_type=MEDinstance.params.process.ih['type'], 
            n_q=MEDinstance.params.process.ih['val'], 
            user_set_min_val=MEDinstance.params.process.user_set_min_value
        )

#### Intensity histogram features

An intensity histogram is generated by discretizing the original intensity distribution into
intensity bins.

In [21]:
int_hist = MEDimage.biomarkers.intensity_histogram.extract_all(
            vol=vol_quant_re
        )

#### Intensity-volume histogram features

The (cumulative) intensity-volume histogram (IVH) of the set of voxel intensities in the ROI
intensity mask describes the relationship between discretized intensity and the fraction of the
volume containing at least intensity the same intensity.

IVH features computation doesn't share the same discretization parameters, so we discretize a second time using the IVH discretization parameters and the discretization interval *wd* should be provided this time.

In [22]:
if MEDinstance.params.process.ivh and 'type' in MEDinstance.params.process.ivh and 'val' in MEDinstance.params.process.ivh:
    if MEDinstance.params.process.ivh['type'] and MEDinstance.params.process.ivh['val']:
        vol_quant_re, wd = MEDimage.processing.discretize(
                vol_re=vol_int_re,
                discr_type=MEDinstance.params.process.ivh['type'], 
                n_q=MEDinstance.params.process.ivh['val'], 
                user_set_min_val=MEDinstance.params.process.user_set_min_value,
                ivh=True
        )
else:
    vol_quant_re = vol_int_re
    wd = 1

Now we can extract the IVH features

In [23]:
int_vol_hist = MEDimage.biomarkers.int_vol_hist.extract_all(
                    MEDimg=MEDinstance,
                    vol=vol_quant_re,
                    vol_int_re=vol_int_re, 
                    wd=wd
        )

#### Texture Features extraction

In this section, for each text scale<sup>1</sup>, for each discretization algorithm<sup>2</sup> and for each grey level<sup>3</sup> we extract the matrix-based features using MEDimageComputeRadiomics methods : 

*Grey level co-occurrence based features (GLCM), grey level run length based features (GLRLM), grey level size zone matrix (GLSZM), grey level distance zone matrix (GLDZM), neighborhood grey tone difference matrix (NGTDM) and neighboring grey level dependence matrix (NGLDM).*

All the processing done in this section is the same as last section, but with different parameters (For example : discretization algorithms may be different for texture features).

After the computation is finished, we update the radiomics structure (update the attributes made to save the results) 

<sup>1</sup> For each time we resample the voxel spacing (In this case we resample the voxel spacing more than one time).

<sup>2</sup> Many discretization algorithm can be given and features will be computed for each one of them.

<sup>3</sup> For each discrete intensity.

**PS**: For GLCM and GLRLM features, merging method may differ from configuration to another (*merged* or *averaged*). You can change the method using the keyworded argument *glcm_merge_method*, *glrlm_merge_method* in the functions *getGLCMfeatures()* and *getGLRLMfeatures()* respectively.

In [24]:
from itertools import product

# Compute radiomics features for each scale text
count = 0
for s in range(MEDinstance.params.process.n_scale):
    # Interpolation
    # Intensity Mask
    vol_obj = MEDimage.processing.interp_volume(
        MEDimage=MEDinstance,
        vol_obj_s=vol_obj_init,
        vox_dim=MEDinstance.params.process.scale_text[s],
        interp_met=MEDinstance.params.process.vol_interp,
        round_val=MEDinstance.params.process.gl_round,
        image_type='image', 
        roi_obj_s=roi_obj_init,
        box_string=MEDinstance.params.process.box_string
    )
    # Morphological Mask
    roi_obj_morph = MEDimage.processing.interp_volume(
        MEDimage=MEDinstance,
        vol_obj_s=roi_obj_init,
        vox_dim=MEDinstance.params.process.scale_text[s],
        interp_met=MEDinstance.params.process.roi_interp,
        round_val=MEDinstance.params.process.roi_pv, 
        image_type='roi', 
        roi_obj_s=roi_obj_init,
        box_string=MEDinstance.params.process.box_string
    )

    # Re-segmentation
    # Intensity mask range re-segmentation
    roi_obj_int = deepcopy(roi_obj_morph)
    roi_obj_int.data = MEDimage.processing.range_re_seg(
        vol=vol_obj.data, 
        roi=roi_obj_int.data,
        im_range=MEDinstance.params.process.im_range
    )
    # Intensity mask outlier re-segmentation
    roi_obj_int.data = np.logical_and(
        MEDimage.processing.outlier_re_seg(
            vol=vol_obj.data, 
            roi=roi_obj_int.data, 
            outliers=MEDinstance.params.process.outliers
        ),
        roi_obj_int.data
    ).astype(int)

    # Compute features for each discretisation algorithm and for each grey-level  
    for a, n in product(range(MEDinstance.params.process.n_algo), range(MEDinstance.params.process.n_gl)):
        count += 1 
        # Preparation of computation :
        MEDinstance.init_tf_calculation(algo=a, gl=n, scale=s)

        # ROI Extraction :
        vol_int_re = MEDimage.processing.roi_extract(
            vol=vol_obj.data, 
            roi=roi_obj_int.data)

        # Discretisation :
        vol_quant_re, _ = MEDimage.processing.discretize(
            vol_re=vol_int_re,
            discr_type=MEDinstance.params.process.algo[a], 
            n_q=MEDinstance.params.process.gray_levels[a][n], 
            user_set_min_val=MEDinstance.params.process.user_set_min_value
        )

        # GLCM features extraction
        glcm = MEDimage.biomarkers.glcm.extract_all(
            vol=vol_quant_re, 
            dist_correction=MEDinstance.params.radiomics.glcm.dist_correction)

        # GLRLM features extraction
        glrlm = MEDimage.biomarkers.glrlm.extract_all(
            vol=vol_quant_re,
            dist_correction=MEDinstance.params.radiomics.glrlm.dist_correction)

        # GLSZM features extraction
        glszm = MEDimage.biomarkers.glszm.extract_all(
            vol=vol_quant_re)

        # GLDZM features extraction
        gldzm = MEDimage.biomarkers.gldzm.extract_all(
            vol_int=vol_quant_re, 
            mask_morph=roi_obj_morph.data)

        # NGTDM features extraction
        ngtdm = MEDimage.biomarkers.ngtdm.extract_all(
            vol=vol_quant_re, 
            dist_correction=MEDinstance.params.radiomics.ngtdm.dist_correction)

        # NGLDM features extraction
        ngldm = MEDimage.biomarkers.ngldm.extract_all(
            vol=vol_quant_re)

        # Update radiomics results class
        MEDinstance.update_radiomics(
                        int_vol_hist_features=int_vol_hist, 
                        morph_features=morph,
                        loc_int_features=local_intensity, 
                        stats_features=stats, 
                        int_hist_features=int_hist,
                        glcm_features=glcm, 
                        glrlm_features=glrlm, 
                        glszm_features=glszm, 
                        gldzm_features=gldzm, 
                        ngtdm_features=ngtdm, 
                        ngldm_features=ngldm
                    )

Finally we print the results

In [25]:
from numpyencoder import NumpyEncoder
from json import dumps

print(dumps(MEDinstance.radiomics.image, indent=4, cls=NumpyEncoder))

{
    "Diagnostics-initial": {
        "scale2": {
            "image_initial_dimX": 204,
            "image_initial_dimY": 201,
            "image_initial_dimz": 60,
            "image_initial_voxDimX": 0.9769999980926514,
            "image_initial_voxDimY": 0.9769999980926514,
            "image_initial_voxDimZ": 3.0,
            "image_initial_meanInt": -266.4704895019531,
            "image_initial_minInt": -1000.0,
            "image_initial_maxInt": 3065.0,
            "roi_initial_Int_dimX": 204,
            "roi_initial_Int_dimY": 201,
            "roi_initial_Int_dimZ": 60,
            "roi_initial_Int_boxBoundDimX": 100,
            "roi_initial_Int_boxBoundDimY": 99,
            "roi_initial_Int_boxBoundDimZ": 26,
            "roi_initial_Morph_boxBoundDimX": 100,
            "roi_initial_Morph_boxBoundDimY": 99,
            "roi_initial_Morph_boxBoundDimZ": 26,
            "roi_initial_Int_voxNumb": 125256,
            "roi_initial_Morph_voxNumb": 125256,
            "roi_

You can compare your results with other teams and check your level of consensus using the CSV provided [here](https://ibsi.radiomics.hevs.ch/assets/IBSI-1-submission-table.xlsx)